In [1]:
import cv2
import mediapipe as mp
import argparse
from pythonosc import udp_client

cap = cv2.VideoCapture(0)
mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

In [3]:
START_SOUND = True
# start the osc
# argparse helps writing user-friendly commandline interfaces
parser = argparse.ArgumentParser()
# OSC server ip: '127.0.0.1'
parser.add_argument("--ip", default='192.168.113.27', help="The ip of the OSC server")
# OSC server port (check on SuperCollider) 57120
parser.add_argument("--port", type=int, default=7400, help="The port the OSC server is listening on")

# Parse the arguments
args = parser.parse_args()

# Start the UDP Client
client = udp_client.SimpleUDPClient(args.ip, args.port)

usage: ipykernel_launcher.py [-h] [--ip IP] [--port PORT]
ipykernel_launcher.py: error: unrecognized arguments: --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"fcd5c4e4-2980-49c3-99be-7a40153fd02c" --shell=9002 --transport="tcp" --iopub=9004 --f=c:\Users\rikyd\AppData\Roaming\jupyter\runtime\kernel-v2-13936xeaic0LQzrzI.json


SystemExit: 2

c:\Users\rikyd\Documents\GitHub\CPAC_Group_7\project-venv\lib\site-packages\IPython\core\interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
while True:
    success, image = cap.read()
    image = cv2.flip(image, 1)
    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(imageRGB)
    center_x = []
    center_y = []
    # checking whether a hand is detected
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks: # working with each hand
            for id, lm in enumerate(handLms.landmark):
                h, w, c = image.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                if(id == 0 or id == 5 or id == 17):
                    center_x.append(cx)
                    center_y.append(cy)
            center_palm_x = int(sum(center_x)/len(center_x))
            center_palm_y = int(sum(center_y)/len(center_y))
            cv2.circle(image, (center_palm_x, center_palm_y), 10, (255, 0 ,255), cv2.FILLED)

            mpDraw.draw_landmarks(image, handLms, mpHands.HAND_CONNECTIONS)
    cv2.imshow("Output", image)
    cv2.waitKey(1)

    if START_SOUND:
        client.send_message("freq", center_palm_x)
	    
        client.send_message("amp", center_palm_y)

    
    keypress = cv2.waitKey(1) & 0xFF		
    if keypress == ord("q"):
		    break

# free up memory
cap.release()
cv2.destroyAllWindows()